# Контрольная работа №1

Время на выполнение 1 час 20 минут.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine

## Задание №1 (3 балла)

Реализовать алгоритм классификации KNN для мультиклассовой классификации с методом `predict_proba` для предсказания вероятностей классов.

Использовать датасет Wine (`load_wine` из `sklearn.datasets`).

Посмотрим на работу sklearn для дальнейшей проверки

In [34]:
from sklearn.datasets import load_wine
from sklearn import metrics
data = load_wine()
X, y = data['data'], data['target']

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [44]:
from sklearn.metrics import accuracy_score, confusion_matrix
preds = model.predict(X_test)
accuracy_score(y_test, preds)

0.6

Собсвенная релизация

In [58]:
class MyKNN:
    def __init__(self, k = 3):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def calculate_euc_distance(self, x, y):
        return np.sqrt(((x - y)**2).sum())

    def calculate_matrix(self, X_test):
        distances = np.zeros((X_test.shape[0], self.X_train.shape[0]))
        for i in range(X_test.shape[0]):
            for j in range(self.X_train.shape[0]):
                distances[i, j] = self.calculate_euc_distance(X_test[i], self.X_train[j])
        return distances

    def predict(self, X_test):
        matrix = self.calculate_matrix(X_test)
        matrix_idx = np.argsort(matrix, axis=1)[:, :self.k]
        res_matrix = np.array([self.y_train[x] for x in matrix_idx])
        return np.array([np.bincount(row).argmax() for row in res_matrix])

    def predict_proba(self, X_test):
        matrix = self.calculate_matrix(X_test)
        matrix_idx = np.argsort(matrix, axis=1)[:, :self.k]

        neighbors = np.array([self.y_train[x] for x in matrix_idx])

        probabilities = []
        for row in neighbors:
            counts = np.bincount(row, minlength=np.unique(self.y_train).size)
            prob = counts / self.k
            probabilities.append(prob)

        return np.array(probabilities)

Посчитаем оценку по accuracy_score

In [59]:
knn = MyKNN(3)
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)
probabilities = knn.predict_proba(X_test)

accuracy_score(y_test, predictions.round(0))

0.6

## Задание №2 (3 балла)

В линейной регрессии оценить влияние признаков на результат.

Использовать датасет <https://archive.ics.uci.edu/dataset/10/automobile> (из файла `imports-85.data`)

In [62]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [67]:
data = pd.read_csv("imports-85.data", header=None, na_values="?")
data.loc[:, data.dtypes == "float64"] = data.loc[:, data.dtypes == "float64"].fillna(
    data.loc[:, data.dtypes == "float64"].mean(axis=0)
)
data.loc[:, data.dtypes == "object"] = data.loc[:, data.dtypes == "object"].fillna("")
data_encoded = pd.get_dummies(data)
data_encoded.columns = data_encoded.columns.astype(str)
X = data_encoded.drop(columns=["25"])
y = data_encoded["25"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=3)


scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

model = LinearRegression().fit(X_train, y_train)
preds = model.predict(X_test)

In [81]:
model.

<bound method BaseEstimator.get_params of LinearRegression()>

In [114]:
coefficients = model.coef_
feature_names = X.columns
coef = np.array([coefficients, feature_names])
coef[0] = np.abs(coef[0])


coef = pd.DataFrame(
    {
        '11': np.abs(coefficients),
        '22':feature_names
    })
coef.sort_values(by='11')

,11,22
1,3.645768,1
35,5.265490,2_volkswagen
43,5.375800,5_two
63,10.275374,15_four
42,14.352494,5_four
...,...,...
10,1768.175885,20
55,1983.641303,14_dohcv
3,2211.412743,10
7,3386.650112,16


## Задание №3 (4 балла)

Изменить решающее дерево под задачу регрессии (минимизация MSE).

Использовать тот же датасет, что и в предыдущем задании.

In [84]:
class MyDecisionTree:
    LEAF = 1
    NON_LEAF = 0

    def __init__(self, max_depth=5, min_samples_split=2) -> None:
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = dict()
        pass

    def fit(self, X_train, y_train):
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        self.fit_node(X_train, y_train, 0, 0)

    def _mse(self, y):
        return np.mean((y - np.mean(y)) ** 2)

    def get_mse_score(self, y_left, y_right):
        total_length = len(y_left) + len(y_right)
        if total_length == 0:
            return 0
        mse_l = self._mse(y_left)
        mse_r = self._mse(y_right)
        return (len(y_left) / total_length) * mse_l + (len(y_right) / total_length) * mse_r

    def find_threshold(self, X, y: np.ndarray):
        min_mse = np.inf
        feature_split = -1
        threshold_split = -1

        for feat in range(X.shape[1]):
            column = X[:, feat]
            unique_values = np.unique(column)
            for value in unique_values:
                left_mask = column <= value
                right_mask = column > value
                y_left = y[left_mask]
                y_right = y[right_mask]
                current_mse = self.get_mse_score(y_left, y_right)
                if current_mse < min_mse:
                    feature_split = feat
                    threshold_split = value
                    min_mse = current_mse

        return feature_split, threshold_split

    def make_final_leaf(self, y, node_id):
        value = np.mean(y)
        self.tree[node_id] = [self.LEAF, value]

    def fit_node(self, X_train, y_train, node_id, depth):
        if depth == self.max_depth - 1 or X_train.shape[0] <= self.min_samples_split:
            self.make_final_leaf(y_train, node_id)
            return

        feature_split, threshold_split = self.find_threshold(X_train, y_train)

        X_left = X_train[X_train[:, feature_split] <= threshold_split]
        y_left = y_train[X_train[:, feature_split] <= threshold_split]
        X_right = X_train[X_train[:, feature_split] > threshold_split]
        y_right = y_train[X_train[:, feature_split] > threshold_split]

        if X_left.shape[0] == 0 or X_right.shape[0] == 0:
            self.make_final_leaf(y_train, node_id)
            return

        self.fit_node(X_left, y_left, 2 * node_id + 1, depth + 1)
        self.fit_node(X_right, y_right, 2 * node_id + 2, depth + 1)

        self.tree[node_id] = [self.NON_LEAF, feature_split, threshold_split]

    def predict_class_one_elem(self, x, node_id):
        if self.tree[node_id][0] == self.LEAF:
            return self.tree[node_id][1]
        else:
            _, feature, split = self.tree[node_id]
            if x[feature] <= split:
                return self.predict_class_one_elem(x, 2 * node_id + 1)
            else:
                return self.predict_class_one_elem(x, 2 * node_id + 2)

    def predict(self, X):
        return np.array([self.predict_class_one_elem(x, 0) for x in X])


In [96]:
my_tree = MyDecisionTree(max_depth=3)

my_tree.fit(X_train, y_train)
preds = my_tree.predict(X_test)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [97]:
accuracy_score(y_test, preds.round(0))

0.0

In [98]:
preds.round(0)

array([ 8912.,  8912.,  8912.,  8912.,  8912., 34820.,  8912.,  8912.,
        8912., 34820.,  8912.,  8912.,  8912., 16340., 34820.,  8912.,
        8912.,  8912.,  8912.,  8912., 16340.])